In [ ]:
import numpy as np
import time
import os

import matplotlib.pyplot as plt
import pandas as pd
import random

import keras
import tensorflow as tf
from tensorflow.keras import models

DEBUG = False

In [ ]:
column_names = ['timestamp','open','close','high','low','volume','adosc','atr','macd','macd_signal','macd_hist','mfi','upper_band','middle_band','lower_band','rsi','difference_low_high','difference_open_close','target']
data_directory = '/home/joren/Coding/cryptodata/Normalized_labelled/'

In [ ]:
## read random binary file
def random_file():
    filenames = []
    for file in os.listdir(data_directory):
        filename = os.fsdecode(file)
        filenames.append(filename)
        
    randomname = filenames[random.randint(0, len(filenames)-1)]
    if randomname.endswith(".bin"): 
        print(f"reading file: {os.path.join(data_directory, randomname)}")
        return os.path.join(data_directory, randomname), randomname

field_info = [
    { "type": np.uint64, "count": 1 },
    { "type": np.double, "count": 17 },
    { "type": np.int64, "count": 1 }
]
BYTES_EIGHT = 8

def read_bin_full_file(file):
    f = open(file, 'rb')
    b = f.read(-1)

    BYTES_TO_READ = 0
    for field in field_info:
        BYTES_TO_READ += BYTES_EIGHT * field["count"]

    data = []
    BYTES_READ = 0
    for i in range(0, int(os.path.getsize(file) / BYTES_TO_READ)):
        row = []

        for idx, field in enumerate(field_info):
            row += np.frombuffer(b, dtype=field["type"], count=field["count"], offset=BYTES_READ).tolist()

            BYTES_READ += BYTES_EIGHT * field["count"]

        data.append(row)
    return np.array(data)

In [ ]:
# train-test split
def df_split(df):  
    X = df.drop(columns=['timestamp','target'], axis=0).to_numpy()
    Y = df['target'].to_numpy()

    X_test = []
    y_test = []
    for i in range(frame_size, X.shape[0]): 
        X_test.append(X[i-frame_size:i])
        y_test.append(Y[i])
    X_test, y_test = np.array(X_test), np.array(y_test)

    return X_test, y_test

In [3]:
## load
model = models.load_model('./models/model1')

## prepare data for prediction
file, filename = random_file()
data = read_bin_full_file(file)
df = pd.DataFrame(data, columns=column_names)
df.fillna(0, inplace=True)
df = df.iloc[240:]
X_test, y_test = df_split(df)

## predict
y_pred = np.array(model.predict(X_test))

y_pred = np.argmax(y_pred, axis=-1, keepdims=True)
y_pred = y_pred.flatten()

print(f"""
Class. report:
{classification_report(y_test, y_pred)}
""")
cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100) 



NameError: name 'models' is not defined

In [ ]:
## plot predicted data against binary data
# https://stackoverflow.com/questions/8409095/set-markers-for-individual-points-on-a-line-in-matplotlib

xs = df['timestamp']
ys = df['close']
plt.plot(xs, ys, '-gD', label='dataplot')
plt.legend()
plt.show()

In [ ]:
## read csv

In [ ]:
## combine csv with binary file across timestamps to get original price candles

In [ ]:
## plot csv data with predictions